# Notebook used to obtain daily stock data information 

In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# URLs to webscrape companies of target
midcap = "http://investsnips.com/list-of-publicly-traded-mid-cap-oil-gas-exploration-and-production-companies/"
largecap = "http://investsnips.com/list-of-publicly-traded-large-cap-oil-gas-exploration-and-production-companies/"

In [3]:
# Function to scrape for midcap companies
def midcap_ls(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    midcap_html = soup.find(class_="et_pb_text_inner").find_all('a')
    company_list = []
    for text in midcap_html:
        company = text.get_text()
        if re.findall("Mid-Cap+", company):
            company_list = []        
            continue        
        company_list.append(company)    
        if re.findall("WPX+", company):
            break
    return company_list

In [4]:
# Function to scrape for largecap companies
def largecap_ls(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    largecap_html = soup.find(class_="et_pb_text_inner").find_all('a')
    company_list = []
    for text in largecap_html:
        company = text.get_text()
        if re.findall("Oil and+", company):
            company_list = []        
            continue        
        company_list.append(company)    
        if re.findall("Pioneer+", company):
            break
    return company_list

In [5]:
# Full list of all the companies
all_companies = midcap_ls(midcap) + (largecap_ls(largecap))

In [6]:
all_companies

['Antero Resources Corporation (AR)',
 'Callon Petroleum Company (CPE)',
 'Centennial Resource Development, Inc. (CDEV)',
 'Chesapeake Energy Corporation (CHK)',
 'Diamondback Energy, Inc. (FANG)',
 'Energen Corporation (EGN)',
 'EQT Corporation (EQT)',
 'Extraction Oil & Gas, Inc. (XOG)',
 'Gulfport Energy Corporation (GPOR)',
 'Jagged Peak Energy Inc. (JAG)',
 'Laredo Petroleum, Inc. (LPI)',
 'Matador Resources Company (MTDR)',
 'MDU Resources Group, Inc. (MDU)',
 'Murphy Oil Corporation (MUR)',
 'Newfield Exploration Company (NFX)',
 'Oasis Petroleum Inc. (OAS)',
 'Parsley Energy, Inc. (PE)',
 'PDC Energy, Inc. (PDCE)',
 'Range Resources Corporation (RRC)',
 'Rice Energy Inc. (RICE)',
 'RSP Permian, Inc. (RSPP)',
 'SM Energy Company (SM)',
 'Southwestern Energy Company (SWN)',
 'Whiting Petroleum Corporation (WLL)',
 'WPX Energy, Inc. (WPX)',
 'Anadarko Petroleum Corporation (APC)',
 'Apache Corporation (APA)',
 'Cabot Oil & Gas Corporation (COG)',
 'Canadian Natural Resources Limit

In [7]:
# Function to pull just stock ticker names
def ticker_name(company_list):
    ticker_ls = []
    for company in company_list:
        start = company.find("(") + 1
        end = company.find(")")
        ticker_ls.append(company[start:end])
    return ticker_ls

In [8]:
# Full list of comapny's stock ticker 
ticker_list = ticker_name(all_companies)

# Next File

In [1]:
# Import Dependecies
import pandas as pd
import time
import quandl
quandl.ApiConfig.api_key = "VDT3_1W-5_X47sL4YjsC"
quandl.ApiConfig.verify_ssl = False

In [2]:
# Crude prices for each day
crude_price = quandl.get('EIA/PET_RWTC_D')

C:\Users\nicho\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [3]:
crude_price_df = crude_price.reset_index()
crude_price_df.to_csv("../data/crudePrice.csv")

# Only run once
## If no files in data folder run
### Only needed to create csv files for each company

In [14]:
# Url to be used to access ticker api data 
url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=full&apikey=BGZ66XLUGR27TO4M"
path = "../data/"
counter = 1

# loop to create csv for each Company
for tick in ticker_list:
    # Will add company of interest to the API and make the request
    url_ticker = url.format(tick)
    response = requests.get(url_ticker)
    # Makes sure that request suceeded
    if response.status_code == 200:
        print("staging")
        ticker_json = response.json()
        # Checks to make sure the API returned actual information
        if next(iter(ticker_json)) == "Error Message":
            print("Error for " + tick)
            pass
        else:
            print("good to go")
            # Formats data into dataframe
            ticker_df = pd.DataFrame(ticker_json['Time Series (Daily)']).\
                                transpose().\
                                reset_index().\
                                rename(columns={"index":"date", "1. open":"open", "2. high":"high", "3. low":"low",
                                                "4. close": "close", "5. adjusted close":"adjusted_close",
                                                "6. volume":"volume", "7. dividend amount": "dividend_amount", 
                                                "8. split coefficient":"split_coefficient"})
            full_path = path + tick +".csv"
            # Saves file as csv
            ticker_df.to_csv(full_path)
            # Counter used to keep track of number of processed items
            counter+=1
            # Conditional to make the process wait to prevent too many allowed request in a minute
            if counter%5 == 1:
                time.sleep(70) # seconds
    

staging
good to go
staging
good to go
staging
good to go
staging
good to go
staging
good to go
staging
Error for EGN
staging
good to go
staging
good to go
staging
good to go
staging
good to go
staging
good to go


KeyError: 'Time Series (Daily)'